In [14]:
import tensorflow as tf
tf.__version__
# tf1.x升级2.x
"""
$ tf_upgrade_v2 --infile test1.x.py --outfile test2.x.py --reportfile report.txt
"""

'\n$ tf_upgrade_v2 --infile test1.x.py --outfile test2.x.py --reportfile report.txt\n'

In [2]:
import pandas as pd
import numpy as np
import operator
import sys
import os

'''
tensorflow.python.framework.errors_impl.InvalidArgumentError: Expect 15 fields but have 0 in record 0
这个错误是因为训练文件或者测试文件的末尾有空行，将空行删除掉即可
'''

'\ntensorflow.python.framework.errors_impl.InvalidArgumentError: Expect 15 fields but have 0 in record 0\n这个错误是因为训练文件或者测试文件的末尾有空行，将空行删除掉即可\n'

In [3]:
train_file = "F:/db/tmp/data/lr/train.csv"
test_file = "F:/db/tmp/data/lr/test.csv"

output_model = "F:/db/tmp/data/wd_test/model"
output_feature_num_file = "F:/db/tmp/data/wd_test/feature_num.txt"
# 需要提前创建好
output_model_serving = "F:/db/tmp/data/wd_test/model_serving"

In [4]:
# 获取wide侧和deep侧特征
def get_feature_column():
    '''
        所有的特征
            age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
        :return
            wide feature column
            deep feature column
    '''
    # 连续型特征
    age = tf.feature_column.numeric_column("age")
    education_num = tf.feature_column.numeric_column("education-num")
    capital_gain = tf.feature_column.numeric_column("capital-gain")
    capital_loss = tf.feature_column.numeric_column("capital-loss")
    hours_per_week = tf.feature_column.numeric_column("hours-per-week")

    # hash
    # 离散型特征先进行hash之后放入wide侧然后再进行embedding最后将embedding得到的结果放入deep侧
    # 因为特征的取值范围不会超过 512 暂时不使用太多的
    work_class = tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size = 512)
    education = tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size = 512)
    marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital-status", hash_bucket_size = 512)
    occupation = tf.feature_column.categorical_column_with_hash_bucket("occupation", hash_bucket_size = 512)
    relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size = 512)

    # 离散化
    # 对 年龄，收入，支出 作离散化
    # 年龄段分割 boundaries
    age_bucket = tf.feature_column.bucketized_column(age, boundaries = [18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
    gain_bucket = tf.feature_column.bucketized_column(capital_gain, boundaries = [0, 1000, 2000, 3000, 10000])
    loss_bucket = tf.feature_column.bucketized_column(capital_loss, boundaries = [0, 1000, 2000, 3000, 5000])

    # 交叉
    # 构建交叉特征
    cross_columns = [
        # 将年龄和收入交叉
        # 年龄分为了9段，收入分为了4段，所以交叉完是36段
        tf.feature_column.crossed_column([age_bucket, gain_bucket], hash_bucket_size = 36),
        # 收入和支出交叉
        tf.feature_column.crossed_column([gain_bucket, loss_bucket], hash_bucket_size = 16),
    ]

    # wide侧特征包括 hash，离散化，交叉
    base_columns = [work_class, education, marital_status, occupation, relationship, age_bucket, gain_bucket, loss_bucket,]
    wide_columns = base_columns + cross_columns

    # deep侧特征包括 连续 hash值的embedding
    deep_columns = [
        age,
        education_num,
        capital_gain,
        capital_loss,
        hours_per_week,
        # 向量的维度选择 9 维，因为 2 ^ 9 是 512，可以涵盖上面的hash
        tf.feature_column.embedding_column(work_class, 9),
        tf.feature_column.embedding_column(education, 9),
        tf.feature_column.embedding_column(marital_status, 9),
        tf.feature_column.embedding_column(occupation, 9),
        tf.feature_column.embedding_column(relationship, 9),
    ]

    return wide_columns, deep_columns

In [5]:
def build_model_estimator(wide_column, deep_column, model_folder):
    '''
    :param wide_column:
    :param deep_column:
    :param model_folder:
    :return:
        2
            模型的实例
            辅助模型导出提供服务的函数
    '''
    # 使用高阶api
    model_es = tf.compat.v1.estimator.DNNLinearCombinedClassifier(
        model_dir = model_folder,
        # wide特征
        linear_feature_columns = wide_column,
        # 优化器(学习率，l2正则化=正则化参数)
        linear_optimizer = tf.compat.v1.train.FtrlOptimizer(0.1, l2_regularization_strength = 1.0),
        # deep特征
        dnn_feature_columns = deep_column,
        # 优化器(学习率，l1正则化=正则化参数, l2正则化=正则化参数)[不同优化器之间的目的都是为了控制参数迭代过程的平稳，每次迭代的幅度不要太大]
        dnn_optimizer = tf.compat.v1.train.ProximalAdagradOptimizer(learning_rate = 0.1, l1_regularization_strength = 0.001, l2_regularization_strength = 0.001),
        # 隐层的维度(4层)[隐层的节点个数决定参数的整体维度]
        dnn_hidden_units = [128, 64, 32, 16],
    )

    # 辅助函数
    # 所有的特征
    feature_column = wide_column + deep_column
    feature_spec = tf.feature_column.make_parse_example_spec(feature_column)
    serving_input_fn = (tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec))

    return model_es, serving_input_fn

In [6]:
def input_fn(data_file, re_time, shuffle, batch_num, predict):
    '''
    :param data_file:
        输入文件
            训练文件或者测试文件
    :param re_time:
        重复采样的次数
    :param shuffle:
        是否要打乱数据 Boolean
    :param batch_num:
        采用随机梯度下降时，多少个样本更新一次参数
    :param predict:
        训练还是测试 Boolean
    :return:
        2
            训练的feature和label
            测试的feature
    '''
    # print(data_file, "data_file")
    # sys.exit()

    _CSV_COLUMN_DEFAULTS = [[0], [''], [0], [''], [0], [''], [''], [''], [''], [''], [0], [0], [0], [''], ['']]

    _CSV_COLUMNS = [
        "age", "workclass", "fnlwgt", "education", "education-num",
        "marital-status", "occupation", "relationship", "race", "sex",
        "capital-gain", "capital-loss", "hours-per-week", "native-country",
        "label"
    ]

    def parse_csv(value):
        columns = tf.io.decode_csv(value, record_defaults = _CSV_COLUMN_DEFAULTS)
        # 字典形式，key是上面的特征名称，value是稀疏列表[0,0,0,0,1,0,0 ...]
        features = dict(zip(_CSV_COLUMNS, columns))
        labels = features.pop("label")
        classes = tf.equal(labels, ">50K")
        return features, classes

    # 预测时不需要返回label
    def parse_csv_predict(value):
        columns = tf.io.decode_csv(value, record_defaults = _CSV_COLUMN_DEFAULTS)
        features = dict(zip(_CSV_COLUMNS, columns))
        labels = features.pop("label")
        return features

    # skip(1) 过滤第一行特征名称以及有 ? 的行，数据清洗
    data_set = tf.data.TextLineDataset(data_file).skip(1).filter(lambda line: tf.not_equal(tf.strings.regex_full_match(line, ".*\?.*"), True))
    # data_set = tf.data.TextLineDataset(data_file).skip(1)

    if shuffle:
        data_set = data_set.shuffle(buffer_size = 30000)
    if predict:
        data_set = data_set.map(parse_csv_predict, num_parallel_calls = 5)
    else:
        data_set = data_set.map(parse_csv, num_parallel_calls = 5)

    # 重复采样
    data_set = data_set.repeat(re_time)
    # 分割测试或者训练
    data_set = data_set.batch(batch_num)
    return data_set

In [7]:
def train_wd_model(model_es, train_file, test_file, model_export_folder, serving_input_fn):
    '''
    :param model_es:
        模型的对象
    :param train_file:
    :param test_file:
    :param model_export_folder:
        模型导出的文件夹
    :param serving_input_fn:
        辅助模型导出提供服务的函数
    :return:
    '''

    '''重复采样'''
    # 利用重复采样来扩充训练集(弥补数据集小)
    total_run = 6
    for index in range(total_run):
        # 模型训练
        # 每一次重复采样10倍
        model_es.train(input_fn = lambda: input_fn(train_file, 10, True, 100, False))
        # 模型评估
        print(model_es.evaluate(input_fn = lambda: input_fn(test_file, 1, False, 100, False)))

    '''不重复采样'''
    # 模型训练
    # model_es.train(input_fn = lambda: input_fn(train_file, 20, True, 100, False))
    # # 模型评估
    # # {'accuracy': 0.8512616, 'accuracy_baseline': 0.7543161, 'auc': 0.9083664, 'auc_precision_recall': 0.77997875, 'average_loss': 0.31840968, 'label/mean': 0.24568394, 'loss': 31.756622, 'precision': 0.7321883, 'prediction/mean': 0.25312397, 'recall': 0.62216216, 'global_step': 6033}
    # print(model_es.evaluate(input_fn = lambda: input_fn(test_file, 1, False, 100, False)))


    # 模型导出（为了给tf_server提供服务的）
    model_es.export_saved_model(model_export_folder, serving_input_fn)

In [8]:
# 得到测试label
def get_test_label(test_file):
    if not os.path.exists(test_file):
        return []
    fp = open(test_file, "r", encoding = "utf-8")
    line_num = 0
    test_label_list = []
    for line in fp:
        if (line):
            if line_num == 0:
                line_num += 1
                continue
            if "?" in line.strip():
                continue
            item = line.strip().split(",")
            # 取最后一行
            label = item[-1].strip()
            if label == ">50K":
                test_label_list.append(1)
            elif label == "<=50K":
                test_label_list.append(0)
            else:
                print(test_file, "error", line)

        # print(test_label_list)
    fp.close()
    return test_label_list

In [9]:
# 准确率
def get_accuracy(predict_list, test_label):
    # 临界值 大于 正样本，小于 负样本
    score_thr = 0.8
    # 预测对的
    right_num = 0
    predict_label_list = []
    for index in range(len(predict_list)):
        predict_score = predict_list[index]
        if predict_score >= score_thr:
            predict_label = 1
        else:
            predict_label = 0
        predict_label_list.append(predict_label)
        if predict_label == test_label[index]:
            # 预测对的
            right_num += 1
    '''
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0] [ 0.  0.  1.  1.  0.  1.  0.  0.  1.  0.]
    accuary: 0.80737
    [0, 0, 0, 1, 0, 1, 0, 0, 1, 1] [ 0.  0.  1.  1.  0.  1.  0.  0.  1.  0.]
    accuary: 0.83778
    '''
    # predict_label_list 预测的结果(可以将测试集去label来模拟真实需要预测的数据)
    # test_label 实际的结果
    print(predict_label_list[:10], test_label[:10])
    total_num = len(predict_list)
    accuracy_score = right_num / total_num
    print("accuracy: %.5f" %(accuracy_score))

In [10]:
# auc
def get_auc(predict_list, test_label):
    '''
    :param predict_list:
        模型预测label
    :param test_label:
        测试label
    # pos 正样本
    auc = (sum(pos_index) - pos_num(pos_num + 1) / 2) / pos_num * neg_num
    '''
    total_list = []
    for index in range(len(predict_list)):
        predict_score = predict_list[index]
        label = test_label[index]
        total_list.append((label, predict_score))
    # 排序
    sorted_total_list = sorted(total_list, key = lambda ele: ele[1])

    # 负样本
    neg_num = 0
    # 正样本
    pos_num = 0
    count = 1
    total_pos_index = 0
    for zuhe in sorted_total_list:
        label, predict_score = zuhe
        if label == 0:
            neg_num += 1
        else:
            pos_num += 1
            # 所有正样本的index + 所处的位置
            total_pos_index += count
        count += 1
    auc_score = (total_pos_index - pos_num * (pos_num + 1) / 2) / (pos_num * neg_num)
    print("auc: %5f" %(auc_score))

In [11]:
# 测评
def test_model_performance(model_es, test_file):
    '''
    :param model_es:
    :param test_file:
    :return:
    '''
    test_label = get_test_label(test_file)
    # 预测
    result = model_es.predict(input_fn = lambda : input_fn(test_file, 1, False, 100, True))
    # 提取每一个样本的预测结果
    predict_list = []
    for one_res in result:
        if "probabilities" in one_res:
            predict_list.append(one_res["probabilities"][1])
    '''
    auc: 0.908438
    预测的                           实际的
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0] [0, 0, 1, 1, 0, 1, 0, 0, 1, 0]
    accuary: 0.81195
    '''
    get_auc(predict_list, test_label)
    get_accuracy(predict_list, test_label)

In [12]:
def run_main(train_file, test_file, model_folder, model_export_folder):
    '''
    :param train_file:
    :param test_file:
    :param model_folder:
        原始模型导出到的文件夹
    :param model_export_folder:
        提供给 TF Server 作 model server服务调用的导出文件夹
        含有通信所需要的PB
    :return:
    '''
    # wide侧和deep侧特征
    wide_column, deep_column = get_feature_column()
    # 构造模型
    model_es, serving_input_fn = build_model_estimator(wide_column, deep_column, model_folder)
    # 训练模型
    '''
    model
        生成的checkpoint文件中記載着最新的model，下次训练时模型会将最新model中的w和b加载到内存中继续训练
            比如123，那么下次就会从456开始
            
    model_serving(提供服务的)
        生成的是秒级的时间戳文件夹，tf server可以直接加载并且提供服务
        *.pb 文件是用来通信的
        variables
            模型训练得到的参数
    '''
    train_wd_model(model_es, train_file, test_file, model_export_folder, serving_input_fn)

    # 利用测试数据集评估模型
    test_model_performance(model_es, test_file)

In [13]:
if __name__ == "__main__":
    run_main(train_file, test_file, output_model, output_model_serving)

W0107 17:27:00.371400  9132 deprecation.py:506] From c:\users\hedy\appdata\local\programs\python\python36\lib\site-packages\tensorflow_core\python\ops\resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
W0107 17:27:00.381400  9132 deprecation.py:323] From c:\users\hedy\appdata\local\programs\python\python36\lib\site-packages\tensorflow_core\python\training\training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W0107 17:27:01.425400  9132 deprecation.py:323] From c:\users\hedy\appdata\local\programs\python\python36\lib\site-packages\ten

{'accuracy': 0.8505312, 'accuracy_baseline': 0.7543161, 'auc': 0.9038848, 'auc_precision_recall': 0.7670498, 'average_loss': 0.32521477, 'label/mean': 0.24568394, 'loss': 32.435326, 'precision': 0.7325843, 'prediction/mean': 0.25261292, 'recall': 0.61675674, 'global_step': 21119}


W0107 17:27:58.207400  9132 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W0107 17:27:58.232400  9132 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.


{'accuracy': 0.8498672, 'accuracy_baseline': 0.7543161, 'auc': 0.9039841, 'auc_precision_recall': 0.76708174, 'average_loss': 0.325002, 'label/mean': 0.24568394, 'loss': 32.41411, 'precision': 0.74120015, 'prediction/mean': 0.24450599, 'recall': 0.59756756, 'global_step': 24136}


W0107 17:28:27.560400  9132 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W0107 17:28:27.584400  9132 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.


...

{'accuracy': 0.85, 'accuracy_baseline': 0.7543161, 'auc': 0.90394866, 'auc_precision_recall': 0.76709276, 'average_loss': 0.32500777, 'label/mean': 0.24568394, 'loss': 32.414684, 'precision': 0.7396076, 'prediction/mean': 0.24533652, 'recall': 0.6010811, 'global_step': 33187}


W0107 17:29:54.433400  9132 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W0107 17:29:54.454400  9132 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.


{'accuracy': 0.8502656, 'accuracy_baseline': 0.7543161, 'auc': 0.90393347, 'auc_precision_recall': 0.7671133, 'average_loss': 0.3250494, 'label/mean': 0.24568394, 'loss': 32.418835, 'precision': 0.73495936, 'prediction/mean': 0.24852645, 'recall': 0.6108108, 'global_step': 36204}


W0107 17:29:57.448400  9132 deprecation.py:323] From c:\users\hedy\appdata\local\programs\python\python36\lib\site-packages\tensorflow_core\python\saved_model\signature_def_utils_impl.py:145: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


auc: 0.903942
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0] [0, 0, 1, 1, 0, 1, 0, 0, 1, 0]
accuracy: 0.81341
